# Metadata driven Extract, Transform And Load process
#### Project Tapestry - https://github.com/mrjonlunn/tapestry
##### Created by: Jon Lunn
##### Version 0.3

In [ ]:
# Add your own parameters here, and call it from a pipeline if you like
file_path = 'Files/Landing/testsource/testentity/NYC1000.csv'
data_source = 'testSource'
entity = 'testEntity'

In [ ]:
%run Standard_NoteBooks

In [ ]:
# Gets the config metadata
config = metadata_loader(data_source, entity)

# Function to create variables from the config file structure
def create_variables(config):
    # Puts config_ in front of the structure items
    prefix = 'config_'
    for key, value in config.items():
        if isinstance(value, dict()):
            create_variables(value, prefix + key + '_')
            print(f'Created variable {prefix}{key} of datatype', type(value).__name__)
            
        else:
            variable_name = prefix + key
            globals()[variable_name] = value
            print(f'Created variable {variable_name} of datatype', type(variable_name).__name__)       

create_variables(config)

In [ ]:
# Loaders based on the file type
# Loader interfaces in loaders note book

if config_file_type == 'csv':
    #df = loadCSV(filePath, header, delimiter, multiLine, customSchema, escape)
    df = loadCSV(filePath, config_file_options_header, config_file_options_delimiter, config_file_options_multiline, config_file_options_escape)

elif config_file_type == 'json':
    #df = loadJson()
    print('not done yet')
elif config_file_type == 'xml':
    #df = loadJson()
    print('not done yet')
elif config_file_type == 'parquet':
    #df = loadJson()
    print('not done yet')
else:
    raise Exception('ERROR: File extension not recognised, or the extension of the file is not the same as the file type defined in the config file')

In [ ]:
# Clean Columns - Removes any special chararters from the column names
if strToBool(config_fileOptions_clean_column_names) == True:
    df = clean_columns(df)

# Check schema of loaded data frame to the setting in the config
if strToBool(config_fileOptions_enforce_schema) == True:
    dataframe_schema = df.schema
    custom_schema = to_dataframe_schema(config_fileOptions_custom_schema)
    if compare_schema(dataframe_schema, custom_schema) == False:
        raise Exception("The data frame schema does not match the config file version")
 

In [ ]:
# Process the dataframe, and get it ready for writing to a table

# Extend the dataframe with the columns to partition on
if config_table_options_raw_partition_type.lower() == 'date':
    df = create_date_partitions(df, config_table_options_raw_partition_date_format, config_table_options_raw_datframe_partition_columns)
elif rawPartitionType.lower() == 'reference':
    print('No partitions required')
# elif rawPartitionType.lower() == 'businessKeys':
#     df = createBusinessKeysPartitions(df, rawPartitionDateFormat, rawDataframePartitionColumns)

# Get dataframe schema

write_schema = df.schema

# Does the table exist?
# Notes: In checking the schema we could do schema merge, however that should be handled by a version. 
# If you are happy using schema merge into Raw remove the raise exception, 
# and create a merge schema flag in the JSON confiig and change the write process to the table for append and merge
if spark.catalog.tableExists(config_table_options_raw_table_name, config_table_options_raw_lakehouse_name) == False:
    create_table_if_not_exists(config_table_options_raw_lakehouse_name, config_table_options_raw_layer_name
                               , config_table_options_raw_table_name, writeSchema, config_table_options_raw_table_partition_columns, onfig_table_options_raw_partition_type)

elif spark.catalog.tableExists(config_table_options_raw_table_name, config_table_options_raw_lakehouse_name) == True and str_to_bool(config_table_options_raw_allow_schema_drift) == False:
    if df_schema_to_table(config_table_options_raw_lakehouse_name, config_table_options_raw_table_name, write_schema) == False:
        raise Exception('There is a schema mis-match between the dataframe and the table')




In [ ]:
if config_table_options_raw_layer_namee != "":
    adjusted_table_name = config_table_options_raw_lakehouse_name + '.' +config_table_options_raw_layer_namee + '_' + config_table_options_raw_table_name
else:
    adjusted_table_name = config_table_options_raw_table_name

try:
    if str_to_bool(config_table_options_raw_structure_only) == False:
        if config_table_options_raw_insert_type.lower() == 'append':
            df.write.format("delta").mode("append").saveAsTable(adjustedTableName)
            
        elif config_table_options_raw_insert_type.lower() == 'overwrite':
            df.write.format("delta").mode("overwrite").saveAsTable(adjustedTableName)

        elif config_table_options_raw_insert_type.lower() == 'merge':

            deltaTable = DeltaTable.forName(spark, adjusted_table_name)
            merge_on_columns = " AND ".join([f"target.{col} = source.{col}" for col in config_table_options_raw_merge_on_columns])
            update_columns = {f"target.{col}": f"source.{col}" for col in config_table_options_raw_merge_update_columns}

            deltaTable.alias("target").merge(
                source=df.alias("source"), 
                condition = merge_on_columns
            ).whenMatchedUpdate(
                set = update_columns
            ).whenNotMatchedInsertAll().execute()
    else:
        print('Structure only flag is set to True, no data has been loaded to the tables')
except:
    move_landing_file(file_path, 'Failure')   
    


In [ ]:
move_landing_file(file_path, 'Success')